In [1]:
import torch
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import os
from tqdm import tqdm
%matplotlib inline

In [2]:
from cook_data import *

In [3]:
model = torch.hub.load('/home/lupin-ya1/notebooks/DATA_INCOME/models/yolov5', 
                       'custom', 
#                        path='/home/lupin-ya1/notebooks/DATA_INCOME/models/yolov5/yolov5s6.pt',
                       path='/home/lupin-ya1/notebooks/DATA_INCOME/models/yolov5/custom_models/KIC_best.pt',
                       source='local')

YOLOv5 🚀 2021-7-29 torch 1.7.0+cu101 CUDA:0 (Tesla P100-PCIE-16GB, 16280.875MB)

Fusing layers... 
Model Summary: 232 layers, 7284276 parameters, 0 gradients
Adding AutoShape... 


In [4]:
# interes_labels = {'person', 'bus'}
# labels = {model.names[i]: '' for i in range(len(model.names)) if model.names[i] in interes_labels}
labels = {model.names[i]: '' for i in range(len(model.names))}
labels

{'0_side': '',
 '100_bills': '',
 '200_bills': '',
 '500_bills': '',
 '1000_bills': '',
 '2000_bills': '',
 '5000_bills': '',
 '0_box_closed': '',
 '0_box': '',
 '100_box_brown': '',
 '200_box_green': '',
 '500_box_violet': '',
 '1000_box_turquoise': '',
 '2000_box_blue': '',
 '5000_box_orange': ''}

In [5]:
batchsize = 16

In [6]:
dpath ='/home/lupin-ya1/notebooks/DATA_INCOME/models/yolov5/data/kic_ds_khab'

In [7]:
for _, dirs, _ in os.walk(dpath):
    for d in sorted(dirs):
        if d[0] == '.':
            continue
        try:
            yoloAnnotations = []
            tmp_path = os.path.join(dpath, d)

            for _, _, files in os.walk(tmp_path):
                imgs = [i for i in files if i[-3:] in ('jpg', 'png', 'bmp')]
                break

            for n in tqdm(range(len(imgs) // batchsize + 1), desc=d):
                imgs_batch = imgs[n * batchsize: (n + 1) * batchsize]
                imgs_ = []
                for img in imgs_batch:
                    imgs_.append(Image.open(os.path.join(tmp_path, img)))
                if len(imgs_) > 0:
                    result = model(imgs_, size=640)
                    yoloAnnotations.extend(result.pandas().xyxy)
 
            fromYolaToVia(tmp_path, imgs, yoloAnnotations, labels, project_name='fromYolaToVia')
        except:
            print('error', os.path.join(multi_path, d), )
    break

26_pack: 100%|██████████| 40/40 [00:15<00:00,  2.60it/s]
